In [1]:
#EP PHYS 512
#Problem Set 2, Question 1

In [2]:
import numpy as np
import scipy.interpolate as sp_int
import matplotlib.pyplot as mpl

In [3]:
#define some typical examples of functions to try out variable integration
def lorentz(x):
    return 1/(1+x**2)

def xpoly(x):
    return -1*x - 0.5*x**2 + 2*x**3

def xpoly_int(x): #subtract from another because of the arbitrary constant
    return -1*x**2/2 - 0.5*x**3/3 + 2*x**4/4

In [4]:
def integrate_step(fun,x1,x2,xold=np.empty(0),yold=np.empty(0),tol=10**-5, count=0, DEBUG=False):
    if DEBUG:
        print('integrating from ',x1,' to ',x2)
    count+=1 #keep track of how many times it's called
    
    points=5
    x=np.linspace(x1,x2,points)
    y = np.zeros(points) #create for holding our y values
    
    if len(xold)!=0 and len(xold)!=0: #first iteration won't have old points passed in
        #I realize hardcoding is a little gross, but computationally the best here because
        #we know that we will have 5 points, so we know which indexes are reused
        #bothering with np.where() etc. is just going to slow us down again
        y[[0,2,4]]=yold #the data points that are the same are reused
        if DEBUG:
            print('reused points are ', y)
            print('x and xold are ', x, xold)
        y[[1,3]]=fun(x[[1,3]]) #only recalculate old points
    else:
        y = fun(x) #calculate all of the points
    if DEBUG:
        print('final y is ', y)
    
    #call two different methods to compare answers of to check good integral or not
    area1=(x2-x1)*(y[0]+4*y[2]+y[4])/6
    area2=(x2-x1)*( y[0]+4*y[1]+2*y[2]+4*y[3]+y[4])/12
    myerr=np.abs(area1-area2)
    
    if myerr<tol:
        return area2, count
    else:
        xm=0.5*(x1+x2)
        a1, c1=integrate_step(fun,x1,xm,xold=x[[0,1,2]],yold=y[[0,1,2]],tol=tol/2, DEBUG=DEBUG)
        a2, c2=integrate_step(fun,xm,x2,xold=x[[2,3,4]],yold=y[[2,3,4]],tol=tol/2, DEBUG=DEBUG)
        return a1+a2, count+c1+c2

In [5]:
#for three different examples demonstrate the variable integrator.

##COSINE TEST
print('TESTING WITH COSINE')
x0=0
x1=np.pi/2
ans, count=integrate_step(np.cos,x0,x1,tol=0.001, DEBUG=False)
print('Integrating cosine from ', x0, x1)
print('Integrated answer is ', ans)
print('True answer would have been ', np.sin(x1)-np.sin(x0))
print('Number of times function was called', count, ' saving ', (count-1)*3, ' function calls reusing points')
print('\n\n\n')


##XPOLY TEST
print('TESTING WITH RANDOM POLYNOMIAL')
x0=0
x1=15
ans, count=integrate_step(xpoly,x0,x1,tol=0.001, DEBUG=False)
print('Integrating xpoly from ', x0, x1)
print('Integrated answer is ', ans)
print('True answer would have been ', xpoly_int(x1)-xpoly_int(x0))
print('Number of times function was called', count, ' saving ', (count-1)*3, ' function calls reusing points')
print('\n\n\n')


##LORENTZ TEST
print('TESTING WITH LORENTZ')
x0=-1
x1=1
ans, count=integrate_step(lorentz,x0,x1,tol=0.001, DEBUG=False)
print('Integrating lorentz from ', x0, x1)
print('Integrated answer is ', ans)
print('True answer would have been ', np.arctan(x1)-np.arctan(x0))
print('Number of times function was called', count, ' saving ', (count-1)*3, ' function calls reusing points')

TESTING WITH COSINE
Integrating cosine from  0 1.5707963267948966
Integrated answer is  1.000008295523968
True answer would have been  1.0
Number of times function was called 3  saving  6  function calls reusing points




TESTING WITH RANDOM POLYNOMIAL
Integrating xpoly from  0 15
Integrated answer is  24637.5
True answer would have been  24637.5
Number of times function was called 1  saving  0  function calls reusing points




TESTING WITH LORENTZ
Integrating lorentz from  -1 1
Integrated answer is  1.5707962512293534
True answer would have been  1.5707963267948966
Number of times function was called 7  saving  18  function calls reusing points


The number of function calls you save depends on how many times the recursive function is called. Each time it's called, three function calls are saved. The polynomial fit is very easy, so few function calls are saved, but for the lorentz function 18 are saved.